In [28]:
import hashlib
import time
import random
from operator import*


# Understanding Blockchain: Proof of Work and Merkle Tree

## Introduction
In this document, we'll explore the fundamental concepts of blockchain technology, focusing on the Proof of Work (PoW) consensus mechanism and the Merkle Tree data structure that underpins the blockchain's transactional integrity. We'll implement these concepts in Python and examine their role in maintaining the security and integrity of a blockchain.

## Proof of Work (PoW)
### Overview
Proof of Work (PoW) is a consensus mechanism used in blockchain networks to achieve agreement on the state of the network. It involves solving a computationally intensive puzzle to validate transactions and create new blocks in the blockchain. PoW is used in popular blockchain networks like Bitcoin and Ethereum.

### Explanation
Imagine you're a miner, and you want to add a block to the blockchain. The block includes a bunch of transactions, and your job is to find a special number called a "nonce" that, when combined with other block data, produces a specific pattern when hashed.

### Finding the Nonce
- You start with the block's data, including the transactions and a previous block's hash.
- You also pick a random number called a nonce and combine it with the block's data.
- Next, you use a computer to hash this combined data. A hash is like a fingerprint for data – it's unique to each set of data.
- If the resulting hash doesn't match the required pattern (usually starting with a certain number of zeros), you try another nonce and repeat the process.
- You keep trying different nonces until you find one that produces a hash with the required pattern.
### Why It's Important
The PoW puzzle is intentionally difficult and time-consuming to solve, but verifying the solution is easy. This ensures that adding a block to the blockchain requires real computational work, making it expensive for someone to tamper with the blockchain's history, but easy for others to verify the work.

### Implementation
We'll begin by implementing the PoW algorithm, including the Block and Blockchain classes.


In [ ]:
class Block:
    def __init__(self, block_id, previous_hash, transactions, timestamp):
        self.block_id = block_id
        self.previous_hash = previous_hash
        self.transactions = transactions
        self.timestamp = timestamp
        self.nonce = 0

    def compute_hash(self):
        return hashlib.sha256((str(self.block_id) + self.previous_hash + str(self.transactions) + str(self.timestamp) + str(self.nonce)).encode()).hexdigest()

    def __str__(self):
        return f"\nBlock ID: {self.block_id}\nPrevious Hash: {self.previous_hash}\nTransactions: {self.transactions}\nTimestamp: {self.timestamp}\nNonce: {self.nonce}"


class Blockchain:
    def __init__(self):
        self.chain = []
        self.difficulty = 4  # Initial difficulty level
        self.target_time_interval = 15  # Target time interval between blocks in seconds
        self.block_count_since_adjustment = 0
        self.total_time_since_adjustment = 0
        # Create genesis block (initial block)
        genesis_block = Block(0, "0", "Genesis", time.time())
        self.add_block(genesis_block)

    def add_block(self, new_block):
        if len(self.chain) > 0:
            new_block.previous_hash = self.chain[-1].compute_hash()

        # Mine the block with adjusted difficulty
        new_block, time_taken = self.mine_block(new_block)

        self.chain.append(new_block)
        print("Block mined successfully.")
        print("Time taken:", time_taken, "seconds")

        self.block_count_since_adjustment += 1
        self.total_time_since_adjustment += time_taken

        # Adjust difficulty every multiple of 10 blocks
        if self.block_count_since_adjustment % 10 == 0:
            self.adjust_difficulty()

    def mine_block(self, block):
        start_time = time.time()
        target = "0" * self.difficulty

        while block.compute_hash()[:self.difficulty] != target:
            block.nonce += 1

        end_time = time.time()
        time_taken = end_time - start_time

        return block, time_taken

    def adjust_difficulty(self):
        average_time = self.total_time_since_adjustment / self.block_count_since_adjustment

        print("Average time taken to mine a block:", average_time, "seconds")

        buffer = 5  # Buffer time interval

        if average_time - 5 < self.target_time_interval:
            self.difficulty += 1
        elif average_time + 5 > self.target_time_interval:
            self.difficulty -= 1

        print("Difficulty adjusted to:", self.difficulty)

        # Reset counters
        self.block_count_since_adjustment = 0
        self.total_time_since_adjustment = 0

### Usage Example
We'll demonstrate the usage of the PoW algorithm by creating a blockchain and adding blocks to it.

In [ ]:
blockchain = Blockchain()

# Create and add more blocks
for i in range(1, 100):
    print("\n---------Adding block #", i, "---------")
    new_block = Block(i, "", "Transaction data " + str(i), time.time())
    blockchain.add_block(new_block)
    print("Block #", i, " added to the blockchain", blockchain.chain[-1])

## Merkle Tree


### Overview
The Merkle Tree is a data structure used to efficiently organize and verify the integrity of transactions within a block in a blockchain.

The Merkle Tree significantly reduces the data needed for proving the validity of a transaction in a blockchain.

In a blockchain, each block contains a large number of transactions. Without a Merkle Tree, verifying the inclusion of a transaction would require providing all the transactions in the block, which could be computationally expensive and inefficient, especially for large blocks.

However, with a Merkle Tree, the proof of inclusion for a transaction requires providing only a subset of hashes from the tree, known as the Merkle path. This path consists of hashes of sibling nodes along the path from the transaction's leaf node to the root of the tree.

The length of the Merkle path is logarithmic in the number of transactions in the block. Specifically, if a block contains 𝑛 transactions, the length of the Merkle path is approximately log₂(𝑛).

For example, if a block contains 1,000 transactions, the Merkle path would require providing only about 10 hashes (assuming a balanced Merkle Tree). This is a significant reduction compared to providing all 1,000 transactions.

Therefore, the Merkle Tree greatly reduces the amount of data needed for proving the validity of a transaction, making blockchain verification more efficient and scalable.

### Implementation
Next, we'll implement the Merkle Tree data structure, including functions for building the tree, retrieving Merkle proofs, and validating proofs.

In [7]:
class MerkleNode:
    def __init__(self, data):
        self.data = data
        self.hash = hashlib.sha256(data.encode()).hexdigest()
        self.left = None
        self.right = None

class MerkleTree:
    def __init__(self, transactions):
        self.root = self.build_tree(transactions)

    def build_tree(self, transactions):
        if len(transactions) == 0:
            return None
        elif len(transactions) == 1:
            return MerkleNode(transactions[0])

        mid = len(transactions) // 2
        left_subtree = self.build_tree(transactions[:mid])
        right_subtree = self.build_tree(transactions[mid:])
        
        root = MerkleNode(left_subtree.hash + right_subtree.hash)
        root.left = left_subtree
        root.right = right_subtree

        return root

    def get_root(self):
        return self.root.hash if self.root else None

    def pretty_print(self, node, prefix="", is_left=True):
        if node is not None:
            self.pretty_print(node.right, prefix + ("│   " if is_left else "    "), False)
            print(prefix + ("└── " if is_left else "┌── ") + node.hash)
            self.pretty_print(node.left, prefix + ("    " if is_left else "│   "), True)

    def get_proof(self, transaction):
        if self.root is None:
            return None, []

        return self._get_proof_helper(self.root, transaction)

    def _get_proof_helper(self, node, transaction):
        if node is None:
            return None, []

        if node.data == transaction:
            return node.hash, []

        left_hash, left_proof = self._get_proof_helper(node.left, transaction)
        if left_hash is not None:
            left_proof.append(node.right.hash)
            return left_hash, left_proof

        right_hash, right_proof = self._get_proof_helper(node.right, transaction)
        if right_hash is not None:
            right_proof.append(node.left.hash)
            return right_hash, right_proof

        return None, []

In [8]:
def validate_proof(merkle_root_hash, transaction_hash, merkle_path):
    if not merkle_path:
        return transaction_hash == merkle_root_hash

    sibling_hash = merkle_path[0]
    computed_hash_1 = hashlib.sha256((transaction_hash + sibling_hash).encode()).hexdigest()
    computed_hash_2 = hashlib.sha256((sibling_hash + transaction_hash).encode()).hexdigest()

    left_path = validate_proof(merkle_root_hash, computed_hash_2, merkle_path[1:])
    right_path = validate_proof(merkle_root_hash, computed_hash_1, merkle_path[1:])

    return left_path or right_path

### Usage Example
We'll demonstrate the usage of the Merkle Tree by constructing a tree from a list of transactions, retrieving a Merkle proof for a transaction, and validating the proof.

In [19]:
transactions = [f"transaction{i}" for i in range(1, 20)]

merkle_tree = MerkleTree(transactions)

print("Merkle Root:", merkle_tree.get_root())
print("\nMerkle Tree:")
merkle_tree.pretty_print(merkle_tree.root)

transaction_to_verify = f'transaction{random.randint(1, 20)}'
print("\nVerifying transaction:", transaction_to_verify)
transaction_hash, merkle_path = merkle_tree.get_proof(transaction_to_verify)
print("\nTransaction Hash:", transaction_hash)
print("Merkle Path Length:", len(merkle_path), "\n","Merkle Path:", merkle_path)

is_valid = validate_proof(merkle_tree.get_root(), transaction_hash, merkle_path)
print("Is Valid:", is_valid)

Merkle Root: 1fee6174e1c18c1121343971627630a08ee0f6686c9707b1da6693fa7139ae50

Merkle Tree:
│                   ┌── 3fd376716c37283599026d83ee37735b98d74b31160ae9fdf471a3a77bee4cc2
│               ┌── 0e44cbee5eb0f9cb478baa8c5a833a039644f6b03737e9b508077ae3358aca16
│               │   └── 6e08d35bf9f37cd3b7248edce4b7093c0cd272ed311d6a3c3765259a4e231510
│           ┌── 4044cd701a75c8d896cd9cdcfa542a4568e488f1fb55d56301337b0f1992ecfc
│           │   └── 188201b43d49a2fe9031415f2053c70da18049a58555fce4c805b8873752a613
│       ┌── 13cb5054b531e984104f288c2932246c4126722bf424bb74f9260b509ac45b55
│       │   │   ┌── e0fd183f2da7e7e023020b293e1a29dbb503852e44b10f1908b1f04f7b86a7b0
│       │   └── f8cfc1e06b95bde47b51de1ad7482a7117a81071946f35c59dd325fa91bf3172
│       │       └── 13dc09b01344bb4c76726ab74cac547cc06a66c509d1731e14ff464563d52af6
│   ┌── 76e94f3377e1201f7b024fc014e31d6af7dd8d1a2f58484459f7f708e934e1f5
│   │   │           ┌── 2a75167b999815b6a4191cb78c9049c19ca82ceb3980dc794f35ed

Here we have a Merkle Tree with 1 000 000 transactions. We will show how to build the tree and how to retrieve the Merkle proof for a specific transaction, and verify the proof with the minimal amount of data needed.

In [22]:
transactions = [f"transaction{i}" for i in range(1, 1000000)]

merkle_tree = MerkleTree(transactions)

print("Merkle Root:", merkle_tree.get_root())

transaction_to_verify = f'transaction{random.randint(1, 1000000)}'
print("\nVerifying transaction:", transaction_to_verify)
transaction_hash, merkle_path = merkle_tree.get_proof(transaction_to_verify)
print("\nTransaction Hash:", transaction_hash)
print("Merkle Path Length:", len(merkle_path), "\n","Merkle Path:", merkle_path)

is_valid = validate_proof(merkle_tree.get_root(), transaction_hash, merkle_path)
print("Is Valid:", is_valid)




Merkle Root: 61a5161747dd55e52e64756ab03df21d44b6d199887099c76c1144279316a1bb

Verifying transaction: transaction98265

Transaction Hash: 29f16626cc045c2afd32cdfbfaf3248a50422c86972486b05e245f2d1603127f
Merkle Path Length: 20 
 Merkle Path: ['b9ce4337083c381dada525c7de3472f3aed9ee15e2d5e990ca950a9247782221', 'b6efca1c84d08b1c6f69a56c2a831e0bd3099f5e04d5627ec359317691d963a1', '94e838adc9f77f69f583cd35079f70dd27f53f231146b91b07f7173eac6d5259', '5112850d20cafe25aba55896b9000bbfb9be79c575d9087edd43654ac3f8b67d', '859f39726755e58107f7dd1a63020b129d43cf12384cbb85c5aa717b0b11fa1b', '944ddc1f69506b159ae47f7e1c32b8d9221824a4a0451e202e2086c41d842b17', '5370d7d81eff4b470b32a48e424ec1af6a8cbb7493209cc392648d45e98fe1b1', '392ecfa23bbdc83443e4fac4392e4a15d6784a7d6f05e488bb8b8f281ba47127', '2fa189fa506a9fafa5d7b699134f4ed52bc91348a853722da2ebac786d134db0', 'a1df6a66aa482130113943654558cebadc24226eecc1fac08063d9b00f8af842', '73a33d8680608fd517a0394a423ff7dcb8cb43f69da03e2cbc28fef1b09fe410', '0845940fd

In this notebook, we created two important parts of blockchain technology: Proof of Work (PoW) and Merkle Trees.

Proof of Work (PoW) is like a puzzle that miners solve to add blocks to the blockchain. It's hard to search for a solution but easy to check. This makes it tough for someone to cheat and proves the validity of the chain.

Merkle Trees help keep transactions in blocks safe. They organize data efficiently, so it's easy to check if a transaction is valid.


Proof of Work (PoW):
- Checking the validity of a hash: O(1), because it's a simple comparison.


Merkle Trees:
- Constructing a Merkle Tree from a list of transactions: O(n), where n is the number of transactions.
- Verifying a transaction in a Merkle Tree: O(log n), where n is the number of transactions, because you need to traverse the tree from the leaf to the root.

# References
Nakamoto, S. (2008) "Bitcoin: A Peer-to-Peer Electronic Cash System." https://bitcoin.org/bitcoin.pdf

Liu, H., Luo, X., Liu, H. and Xia, X. (2021) "Merkle Tree: A Fundamental Component of Blockchains", International Conference on Electronic Information Engineering and Computer Science (EIECS), pp. 556-561, doi: 10.1109/EIECS53707.2021.9588047. https://ieeexplore.ieee.org/document/9588047